# QSVM

In [2]:
import pandas as pd
from qiskit import Aer
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel, FidelityQuantumKernel
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [3]:
df = pd.read_csv('datasets/custom_dataset.csv', index_col=0)
df.head(3)

,feature_1,feature_2,label
0,3.266799,3.266799,0.0
1,3.104821,3.104821,0.0
2,3.277766,2.722234,0.0


In [4]:
feature_dimension = df.shape[1] - 1
print(f"Feature dimension: {feature_dimension}")

Feature dimension: 2


In [5]:
# Split the data into training and test sets
train = df.sample(frac=0.75, random_state=42)
test = df.drop(train.index)

# Separate the features from the labels
train_features = train.copy()
test_features = test.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

In [6]:
algorithm_globals.random_seed = 42
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), shots=1024)

In [17]:
feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='sca')
# kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
kernel = FidelityQuantumKernel(fidelity=ComputeUncompute(sampler=Sampler()))
svc = SVC(kernel=kernel.evaluate)

In [18]:
svc.fit(train_features, train_labels)
score = svc.score(test_features, test_labels)
print(f"QSVM score: {score}")

QSVM score: 0.728


In [19]:
scores = cross_val_score(svc, train_features, train_labels, cv=10)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[0.68421053 0.68421053 0.68421053 0.68421053 0.68421053 0.7027027
 0.7027027  0.7027027  0.7027027  0.67567568]
0.69 accuracy with a standard deviation of 0.01


In [10]:
classical_svc = SVC()
classical_svc.fit(train_features, train_labels)
score = svc.score(test_features, test_labels)

/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [11]:
print(f"Classical SVC score: {score}")

Classical SVC score: 0.728


In [12]:
scores = cross_val_score(classical_svc, train_features, train_labels, cv=10)
scores

array([0.68      , 0.69333333, 0.76      , 0.74666667, 0.68      ])

In [13]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.71 accuracy with a standard deviation of 0.03
